#### Import the required Libraries 

In [ ]:
#Data handling packages
import pandas as pd

#Data processing packages
import numpy as np

#Visualization packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Additional libraries 
import re
import string
import time
import os

#NLP based libraries
from wordcloud import WordCloud, STOPWORDS
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
import googletrans
from googletrans import Translator


#Preprocessing and model building libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier


#framework 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
data= pd.read_csv("C:\\Users\\Riyaz Mehendi\\OneDrive - CriticalRiver Technologies Pvt. Ltd\\Desktop\\MVP Incident Management\\pim-assignment-group-prediction\\data\\processed\\final_processed_data_1_EBS.csv")

In [ ]:
data = data.replace(['Desktop Services UK', 'Desktop Services Brazil', 'Desktop Services USA Central','Desktop Services India','Desktop Services USA West','Desktop Services Germany','Desktop Services Singapore','Desktop Services USA East',
 'Desktop Services Mexico','Desktop Services China','Desktop Services Canada','Desktop Services France','Desktop Services Romania','Desktop Services Thailand'], 'Desktop Services')


In [ ]:
data.info()

In [ ]:
data=data.drop(['Opened','Resolved',
       'Business resolve time', 'name_string_validation', 'langue',
       'translated_text','string_length'], axis = 1)

In [ ]:
target_names = data['Configuration item'].unique()

In [ ]:
target_names=list(target_names)

In [ ]:
len(target_names)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.replace(np.nan, 'Others')

In [ ]:
data.isnull().sum()

In [ ]:
data['cleaned_name']=data['cleaned_name'].astype(str)

Encoding the categorical variables

In [ ]:
data.replace({'1 - High': 1, '2 - Medium': 2,'3 - Low':3},inplace=True)

In [ ]:
data.replace({'1 - Immediate': 1, '2 - Important': 2,'3 - Standard':3},inplace=True)

In [ ]:
data['Assignment group'].unique()

In [ ]:
data.replace({'Messaging':0,'EBS - CRM Technical':1, 'Core IT Infrastructure':2, 'Helpdesk Team':3,
       'Front Office Applications':4, 'Desktop Services':5, 'EBS - CRM':6,
       'EBS - Finance':7, 'EBS - DBA':8, 'Oracle Apex':9,
       'EBS - Order Management':10, 'Oracle OIM/OID':11, 'Data Warehouse AM':12,
       'Active Directory':13, 'OSP Notes Support':14, 'Web Services':15,
       'GNTS -- L1 Global Network & Telecom Services':16, 'HRIS':17,
       'RPA Development & Support':18, 'Enterprise IT Infrastructure':19,
       'Service Now':20, 'Genesys Support Group':21, 'IT - Wireless':22,
       'SME - RevPro':23, 'DSG':24, 'InfoSec Operations':25, 'EBS - Procurement':26,
       'EBS - Agile':27, 'Hyperion':28, 'IT - Expandium':29, 'Linux Wireless ':30,
       'Custom Apps':31, 'AvComm Applications ':32, 'EBS - Markview':33,
       'ViaviCentral Team':34, 'R12 Responsibilities & Data Conversion':35,
       'EBS - Manufacturing':36, 'OSP Tableau Support':37, 'NSE QE Product':38,
       'R&D IT Infrastructure':39, 'Oracle: SOA':40,
       'Trend Micro Agent support':41, 'ESAP Support':42, 'SQL Server DBA':43,
       'OSP Other MFG Support':44, 'PGP Desktop':45, 'SCM Support ':46,
       'PRM-Business Process Users':47, 'EBS - Human Resources':48, 'IT Audit':49,
       'Taleo Admin':50, 'Ricoh':51, 'OSP MES Support':52,
       'Application Support EMEA':53, 'OSP Barcode Printing Support':54,
       'EBS - Data Management':55, 'NSE Tableau Support':56,
       'EBS - Finance Technical':57, 'Sabrix Support':58,
       'Data Center Infrastructure':59, 'Web Marketing':60,
       'InfoSec Incident Response':61, 'EBS - Markview Technical ':62,
       'EBS - Human Resources Technical ':63, 'SharePoint':64,
       'Licensing Portal':65, 'HP Quality Support':66, 'MES Support':67,
       'NSE Operations':68},inplace=True)

In [ ]:
def EncodeSingleColumn(data):
    le = preprocessing.LabelEncoder()
    le.fit(data)
    return le.transform(data)

In [ ]:
def EncodeColumnsCollection(data, columnsToEncrypt):
    #for column in columnsToEncrypt:
    data[columnsToEncrypt] = EncodeSingleColumn(data[columnsToEncrypt])
    return data

In [ ]:
data['Configuration item']=data['Configuration item'].astype(str)

In [ ]:
# Select columns for encryption
#columnsToEncode = ['Configuration item','Subcategory','Category']

# Encrypt data for each of selected columns
df_encoded = EncodeColumnsCollection(data, 'Configuration item')

In [ ]:
df_encoded

In [ ]:
df_encoded=df_encoded.drop(['Number','Name','Subcategory','Category','cleaned_name'], axis = 1)

In [ ]:
df_encoded.describe()

#### Text Embedding using USE technique

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

In [ ]:
embedded_desc1 = model(data['cleaned_name'])

In [ ]:
embedded_desc1

In [ ]:
#converta array into dataframe
embedded_desc1=pd.DataFrame(embedded_desc1)
embedded_desc1.head()

In [ ]:
new_data = pd.concat([embedded_desc1, df_encoded], axis=1)

In [ ]:
new_data

Split the Dataset into Train and Test

In [ ]:
y= new_data['Assignment group']

In [ ]:
X= new_data.drop(['Assignment group'], axis = 1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0,stratify=y)
print(len(x_train))
print(len(x_test))

In [ ]:
y_train.value_counts()

#### Model Building

In [ ]:
XGB_model= XGBClassifier().fit(x_train,y_train)
y_pred1 = XGB_model.predict(x_test)

In [ ]:
###### Classification report
print('\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred1))